# Creating Output Rasters

The following script converts Raven model output into raster files readily accessible to standard GIS software (e.g., ArcGIS, QGIS). In addition, simulated long-term average recharge rates are converted into MODFLOW-ready files to simulate the groundwater system (in steady-state) under the Raven-simulated recharge regime.

## Load Python packages and import data

In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import shapefile
from PIL import Image, ImageDraw

### Set model output directory, user settings

> Users must change the following input according to the model they wish to convert.

In [2]:
ravenoutputdir = '../model_baseline/output/' # directory of Raven output
modflowdir = '../MODFLOW/'
hrufp = '../model_baseline/Raven2025-hruid.bil' # HRU reference map, produced using update_landuse.ipynb
swsfp = '../GIS/CH_Subs_Raven2025.shp' # Raven sub-basin polygons
mdlprfx = 'Raven2025' # model short-name prefix

grid definition

In [3]:
# (keep consistent with input raster definition)
nrows = 719
ncols = 726
xul = 1323330
yul = 11906070
cs = 60. # grid cell width

Raven to MODFLOW mapping

In [4]:
mapfp = '../scripts/supp/Raven2025_to_SHModel-CH_310_map.pkl' # used to map Raven HRUs to MODFLOW cells

## Import HRU ID raster

In [5]:
# Reads integer rasters
def readIntRaster(fp):
    aa = np.fromfile(fp,np.int32)
    if len(aa) == nrows*ncols:
        x = dict(zip(np.arange(nrows*ncols),aa))
    elif len(aa) == nrows*ncols/2:
        aa = np.fromfile(fp,np.int16)
        x = dict(zip(np.arange(nrows*ncols),aa))
    else:
        print(" ** Warning, unknown raster size {}",fp)
        return None
    return x

In [6]:
xhru = readIntRaster(hrufp) # hruid to MODFLOW cell cross-reference

## Import Sub-Basins

- Collect raster cells contained within each sub-basin

In [7]:
def polygonToCellIDs(pts, nrows=719, ncols=726, xul=1323330, yul=11906070, cs=60):
    img = Image.new('L', (ncols, nrows), 0)
    offset = [xul, yul-nrows*cs]
    pixelpoly = [((x-offset[0])/cs, nrows-(y-offset[1])/cs) for x,y in pts]
    ImageDraw.Draw(img).polygon(pixelpoly, outline=1, fill=1)
    mask = np.array(img)
    return np.arange(nrows*ncols)[mask.reshape(nrows*ncols)>0]    

def loadSWS(fp):
    sf = shapefile.Reader(fp)
    geom = sf.shapes()
    attr = sf.records()
    xr = dict()
    for i in range(len(geom)):
        sid = int(attr[i].SubId)
        xr[sid] = polygonToCellIDs(geom[i].points)
    return xr

In [8]:
xsb = loadSWS(swsfp) # sub-basin to cell cross reference

## Import Model Outputs

- Precipitation
- Total actual evapotranspiration (AET)
- Runoff
- Groundwater recharge
- Infiltration
- Baseflow

### Model output file read function

In [ ]:
def readRaven(fp, isaccumulated, warmup=1200):
    # load data
    df = pd.read_csv(fp, skiprows=1, parse_dates=['month'], date_format="%Y-%m")
    df = df.iloc[:,:-1] # drop last column (..a Raven quirk)
    df = df.drop(columns='time')

    # change accumulation to discrete monthly values
    if isaccumulated:
        df2 = df.diff()
        df2 = df2.fillna(df)
        df2['month'] = df['month']
        df = df2

    df = df.iloc[warmup//30:,:] # drop warmup period

    # rename columns
    cols = [w.replace('mean.','').replace('cumulsum.','') for w in list(df.columns)]
    cols[1] = '0'
    df.columns = cols
    df.index=df['month']
    df = df.drop(columns='month')


    # df = df.groupby(['month']).mean().sum() # long-term mean
    df = df.mean(axis=0)*12
    return {int(k)+1:v for k,v in df.items()}


In [10]:
dprecip = readRaven(ravenoutputdir+mdlprfx+'_PRECIP_Monthly_CumulSum_ByHRU.csv',False)                                 # precipitation
daet = readRaven(ravenoutputdir+mdlprfx+'_AET_Monthly_CumulSum_ByHRU.csv',False)                                       # evapotranspiration
dro = readRaven(ravenoutputdir+mdlprfx+'_RUNOFF_Monthly_CumulSum_ByHRU.csv',False)
                                      
dimpro = readRaven(ravenoutputdir+mdlprfx+'_BETWEEN_PONDED_WATER_AND_SURFACE_WATER_Monthly_Average_ByHRU.csv',True)    # impervious runoff
drecharge = readRaven(ravenoutputdir+mdlprfx+'_BETWEEN_SOIL[1]_AND_SOIL[2]_Monthly_Average_ByHRU.csv',True)            # recharge
dinfilt =  readRaven(ravenoutputdir+mdlprfx+'_BETWEEN_PONDED_WATER_AND_SOIL[0]_Monthly_Average_ByHRU.csv',True)        # infiltration
dbasflw = readRaven(ravenoutputdir+mdlprfx+'_BETWEEN_SOIL[2]_AND_SURFACE_WATER_Monthly_Average_ByHRU.csv',True)        # baseflow

dtopslor = readRaven(ravenoutputdir+mdlprfx+'_SOIL[0]_Monthly_Average_ByHRU.csv',True)                                 # topsoil/vadose zone storage
dfastslor = readRaven(ravenoutputdir+mdlprfx+'_SOIL[1]_Monthly_Average_ByHRU.csv',True)                                # fast storage
dslowslor = readRaven(ravenoutputdir+mdlprfx+'_SOIL[2]_Monthly_Average_ByHRU.csv',True)                                # groundwater storage

def pm(d): return np.average(list(d.values()))
print("""Model water-balance (mm/yr):
    Precipitation: {:5.0f}
    Total ET:      {:5.0f}
    Runoff:        {:5.0f}
    Residual:      {:5.1f}
      
    Runoff:
      Quick flow:  {:5.1f}
      Baseflow     {:5.1f}
      Recharge     {:5.1f}
""".format(pm(dprecip),pm(daet),pm(dro),pm(dprecip)-pm(daet)-pm(dro),pm(dro)-pm(dbasflw),pm(dbasflw),pm(drecharge)))

Model water-balance (mm/yr):
    Precipitation:   813
    Total ET:        455
    Runoff:          364
    Residual:       -6.4
      
    Runoff:
      Quick flow:  196.2
      Baseflow     168.3
      Recharge     168.3



## Map Model Outputs to Raster Files

In [11]:
def dictToNp(dat, nr, nc, nodata=-9999.):
    def crc(c): # cell to row-col
        i = int(c/nc)
        j = c - i*nc
        return (i,j)    

    a = np.full((nr,nc),nodata,dtype=np.float32)
    for cid,v in dat.items(): a[crc(cid)] = np.float32(v)
    return a


def saveBinaryFloat(fp, dat):
    fn, _ = os.path.splitext(fp)
    with open(fn+'.hdr', 'w') as f: # save header
        f.write('BYTEORDER      I\n')
        f.write('LAYOUT         BIL\n')
        f.write('NROWS          '+str(nrows)+'\n')
        f.write('NCOLS          '+str(ncols)+'\n')
        f.write('NBANDS         1\n')
        f.write('NBITS          32\n')
        f.write('BANDROWBYTES   '+str(nrows*4)+'\n')
        f.write('TOTALROWBYTES  '+str(nrows*4)+'\n')
        f.write('PIXELTYPE      FLOAT\n')
        f.write('ULXMAP         '+str(xul+cs/2)+'\n') # The x-axis map coordinate of the center of the upper-left pixel.
        f.write('ULYMAP         '+str(yul-cs/2)+'\n') # The y-axis map coordinate of the center of the upper-left pixel.
        f.write('XDIM           '+str(cs)+'\n')
        f.write('YDIM           '+str(cs)+'\n')
        f.write('NODATA         -9999\n')

    dat = dat.copy()
    if type(dat)==dict: dat = dictToNp(dat,nrows,ncols)
    if os.path.exists(fp): os.remove(fp)
    dat.tofile(fp) # always saved in C-order (row-major)    

Water Balance

$$ P=EA+Q_\text{UZ}+G + \Delta S $$

where $P$ is precipitation, $EA$ is total evapotranspiration, $Q_\text{UZ}$ is runoff, $G$ is groundwater recharge, and $\Delta S$ is the change in surface storage, calculated as the residual.

In [12]:
pre = dict()
aet = dict()
roff = dict()
rch = dict()
delsto = dict()
rsid = dict()
wbal = 0
for c,h in xhru.items():
    if h==-9999: continue
    pre[c]=dprecip[h]
    aet[c]=daet[h]
    roff[c]=dro[h]-dbasflw[h]
    rch[c]=drecharge[h]
    delsto[c]=dtopslor[h]+dfastslor[h]+dslowslor[h]
    if h<5: continue
    rsid[c]=pre[c]-aet[c]-roff[c]-rch[c] # water balance residual
    # rsid[c]=pre[c]-aet[c]-dro[h] # HRU residual
    # rsid[c]=pre[c]-dimpro[h]-dinfilt[h] # surface residual
    # rsid[c]=drecharge[h]-dbasflw[h]-dslowslor[h] # groundwater storage reservoir residual
    wbal += rsid[c]

wbal /= len(xhru)
print("""Model water-balance residuals (mm/yr):
    Residual:               {:7.1f}
    min residual:           {:7.1f}
    max residual:           {:7.1f}
    water left in storage:  {:7.1f}
""".format(wbal,min(rsid.values()), max(rsid.values()), sum(delsto.values())/len(xhru)))

Model water-balance residuals (mm/yr):
    Residual:                  -2.1
    min residual:              -6.8
    max residual:              -1.7
    water left in storage:     -1.5



save waterbalance components to rasters:

In [13]:
saveBinaryFloat(ravenoutputdir+mdlprfx+'_longterm.precipitation.bil',pre)
saveBinaryFloat(ravenoutputdir+mdlprfx+'_longterm.evapotranspiration.bil',aet)
saveBinaryFloat(ravenoutputdir+mdlprfx+'_longterm.runoff.bil',roff)
saveBinaryFloat(ravenoutputdir+mdlprfx+'_longterm.recharge.bil',rch)
saveBinaryFloat(ravenoutputdir+mdlprfx+'_longterm.residual.bil',rsid)

> Rasters are exported to Raven model output directory: `ravenoutputdir`.

### CH sub-watershed summaries

Long-term averages, exported to csv file, by sub-watershed ID in mm/yr

In [14]:
sp, sa, sr, sg = dict(), dict(), dict(), dict(),
for sid, cids in xsb.items():
    sp[sid] = 0.
    sa[sid] = 0.
    sr[sid] = 0.
    sg[sid] = 0.
    for c in cids:
        if not c in pre: continue
        sp[sid] += pre[c]
        sa[sid] += aet[c]
        sr[sid] += roff[c]
        sg[sid] += rch[c]
    sp[sid] /= len(cids)
    sa[sid] /= len(cids)
    sr[sid] /= len(cids)
    sg[sid] /= len(cids)        

df = pd.DataFrame([sp, sa, sr, sg]).T.rename(columns={0:'precipitation',1:'evapotranspiration',2:'runoff',3:'recharge'})
df.index.rename('swsID', inplace=True)
print(df)
df.to_csv(ravenoutputdir+mdlprfx+'_SWS-waterbalance.csv')

       precipitation  evapotranspiration      runoff    recharge
swsID                                                           
1         743.300445          441.976231  190.316996  114.681241
2         753.513795          410.401293  191.032997  155.514590
3         770.286839          456.468372  206.997405  110.714519
4         819.181608          496.421297  226.252974  100.992670
5         767.862204          410.451913  272.706212   88.808675
...              ...                 ...         ...         ...
66        811.340964          476.862069   68.212468  276.141311
68        788.632519          474.948261  152.538042  168.087107
69        803.967470          435.687930  803.967470    0.000000
70        830.484919          484.408007  235.386243  114.769966
71        815.443509          487.787715  187.535352  144.227138

[70 rows x 4 columns]


## Create MODFLOW RCH file

The remaining code block uses only the groundwater recharge estimates to produce:

1. MODFLOW recharge input file (_*RCH_) 
2. Significant Groundwater Recharge Areas (SGRAs) exported as a raster

First, defined MODFLOW grid definition of `SHModel-CH_310-NWT`:

In [15]:
gwNrow = 885
gwNcol = 890

In [16]:
with open(mapfp,'rb') as f: gmap = pickle.load(f)
grch, wrch = dict(), dict()
for c,ws in gmap.items():
    for cRaven,w in ws.items():
        if cRaven==-1: continue
        if c in grch:
            grch[c]+=w*rch[cRaven]
            wrch[c]+=w
        else:
            grch[c]=w*rch[cRaven]
            wrch[c]=w

meanrch = sum(grch.values()) / len(grch) 
for c in gmap:
    if c in grch: 
        grch[c]/=wrch[c]
    else:
        grch[c]=meanrch # infill with mean recharge where Raven does not cover MODFLOW

arch = dictToNp(grch, gwNrow, gwNcol, 0)
# arch.tofile(modflowdir+mdlprfx+ ".rch.bil")
hdr = '         3        49   NRCHOP,IRCHCB\n    0    0   INRECH,INIRCH\nINTERNAL     3.1689E-11 (890E11.3)   -1   Longterm groundwater recharge from {}'.format(mdlprfx)
np.savetxt(modflowdir+mdlprfx+ ".rch", arch, header=hdr, comments='', fmt='%10.3e', delimiter=' ')


## Build SGRAs

In [17]:
# mean = np.mean(rch.values)
sgra = np.array(list(rch.values()))
rchavg = sgra.mean()
print('SGRA threshold: {:.1f} mm/yr'.format(rchavg * 1.15)) # mean recharge
sgra = dict()
for c,r in rch.items():
    if r >= rchavg * 1.15:
        sgra[c]=1
    else:
        sgra[c]=0
saveBinaryFloat(ravenoutputdir+mdlprfx+'_longterm.recharge-SGRA.bil',sgra)

SGRA threshold: 156.8 mm/yr
